# Data analysis of the fantasy premier league

### Goal of Analysis: Use data science to get the best possible team for the next gameweek as well as suggest the best possible transfers according to the available resources.


In [74]:
import numpy as np
import pandas as pd
import requests

In [75]:
FPL_URL = "https://fantasy.premierleague.com/api/bootstrap-static/"

In [76]:
r = requests.get(FPL_URL).json()

In [77]:
r.keys()

dict_keys(['events', 'game_settings', 'phases', 'teams', 'total_players', 'elements', 'element_stats', 'element_types'])

In [78]:
elements_df=pd.DataFrame(r.get('elements'))
element_types_df=pd.DataFrame(r['element_types'])
teams_df=pd.DataFrame(r['teams'])

In [79]:
elements_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Data columns (total 88 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   chance_of_playing_next_round          503 non-null    float64
 1   chance_of_playing_this_round          501 non-null    float64
 2   code                                  740 non-null    int64  
 3   cost_change_event                     740 non-null    int64  
 4   cost_change_event_fall                740 non-null    int64  
 5   cost_change_start                     740 non-null    int64  
 6   cost_change_start_fall                740 non-null    int64  
 7   dreamteam_count                       740 non-null    int64  
 8   element_type                          740 non-null    int64  
 9   ep_next                               740 non-null    object 
 10  ep_this                               740 non-null    object 
 11  event_points       

In [80]:
elements_df.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,NaN,NaN,84450,-1,1,-1,1,1,3,6.6,...,238,138,77,27,58,24,102,36,1.06,0.42
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,...,625,320,487,181,387,169,205,63,0.81,0.00
2,NaN,NaN,156074,0,0,-3,3,0,2,0.6,...,602,170,334,117,421,157,432,163,0.00,0.00
3,100.0,75.0,167199,0,0,-3,3,0,3,4.0,...,314,191,173,66,134,59,260,84,1.06,0.56
4,100.0,100.0,184029,1,-1,5,-5,4,3,10.6,...,43,21,35,15,9,3,9,5,1.07,0.45


In [81]:
elements_df.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round', 'code',
       'cost_change_event', 'cost_change_event_fall', 'cost_change_start',
       'cost_change_start_fall', 'dreamteam_count', 'element_type', 'ep_next',
       'ep_this', 'event_points', 'first_name', 'form', 'id', 'in_dreamteam',
       'news', 'news_added', 'now_cost', 'photo', 'points_per_game',
       'second_name', 'selected_by_percent', 'special', 'squad_number',
       'status', 'team', 'team_code', 'total_points', 'transfers_in',
       'transfers_in_event', 'transfers_out', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'minutes', 'goals_scored',
       'assists', 'clean_sheets', 'goals_conceded', 'own_goals',
       'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards',
       'saves', 'bonus', 'bps', 'influence', 'creativity', 'threat',
       'ict_index', 'starts', 'expected_goals', 'expected_assists',
       'expected_goal_involvements', 'expected_goals_con

In [82]:
slim_elements_df = elements_df[['second_name','team','element_type','selected_by_percent','now_cost','minutes','transfers_in','value_season','total_points']]

In [83]:
slim_elements_df.head()

,second_name,team,element_type,selected_by_percent,now_cost,minutes,transfers_in,value_season,total_points
0,Xhaka,1,3,2.8,49,1706,810283,16.5,81
1,Elneny,1,3,0.7,41,111,196138,1.5,6
2,Holding,1,2,0.1,42,16,6873,1.4,6
3,Partey,1,3,0.5,47,1450,96633,11.5,54
4,Ødegaard,1,3,34.7,70,1598,4605702,17.0,119


In [89]:
element_types_df.head()

,id,plural_name,plural_name_short,singular_name,singular_name_short,squad_select,squad_min_play,squad_max_play,ui_shirt_specific,sub_positions_locked,element_count
0,1,Goalkeepers,GKP,Goalkeeper,GKP,2,1,1,True,[12],77
1,2,Defenders,DEF,Defender,DEF,5,3,5,False,[],250
2,3,Midfielders,MID,Midfielder,MID,5,2,5,False,[],321
3,4,Forwards,FWD,Forward,FWD,3,1,3,False,[],92


In [90]:
positions={1:'Goalkeeper',2:'Defender',3:'Midfielder',4:'Foward'}

In [98]:
slim_elements_df['position']=slim_elements_df['element_type']

C:\Users\tripp\AppData\Local\Temp\ipykernel_14688\4018534799.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  slim_elements_df['position']=slim_elements_df['element_type']
